<a href="https://colab.research.google.com/github/srikanthmandava9/Srikant_INFO5731_Spring2021/blob/master/In_class_exercise_06.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **The sixth in-class-exercise (20 points in total, 3/2/2021)**

## **1. Rule-based information extraction (10 points)**

Use any keywords related to data science, natural language processing, machine learning to search from google scholar, get the **titles** of 100 articles (either by web scraping or manually) about this topic, define a set of patterns to extract the research questions/problems, methods/algorithms/models, datasets, applications, or any other important information about this topic. 

In [6]:

# Write your code here
import re, numpy as nump, pandas as panda
import urllib.request
from bs4 import BeautifulSoup
import pandas as panda
import requests
from itertools import cycle
import traceback
from lxml.html import fromstring
import string 
import nltk 
import spacy 
import numpy as nump 
import math 
from tqdm import tqdm 
from spacy.matcher import Matcher 
from spacy.tokens import Span 
from spacy import displacy 

panda.set_option('display.max_colwidth', 200)
nlp = spacy.load("en_core_web_sm")  

#Retrieving proxies from URL to handle error 429-'Too many requests from the user' to google scholar url
def get_proxies():
    url = 'https://free-proxy-list.net/'
    response = requests.get(url)
    parser = fromstring(response.text)
    proxies = set()
    for i in parser.xpath('//tbody/tr')[:10]:
      if i.xpath('.//td[7][contains(text(),"yes")]'):
        #Grabbing IP and corresponding PORT
        proxy = ":".join([i.xpath('.//td[1]/text()')[0], i.xpath('.//td[2]/text()')[0]])
        proxies.add(proxy)
    return proxies

#Retrieving article_items
def getTitles(soup):
  articles = soup.find("div", {"id": "gs_res_ccl_mid"})
  article_items = articles.find_all(class_ = "gs_r gs_or gs_scl")
  list_articles = []
  for each in article_items:
    list_articles.append(each.find(class_ = "gs_ri"))
  return list_articles

#Retrieving artictle titles and storing into a list of dicts
def getArticle_dict(articles):
  art_list = []
  for each in articles:
    art_list.append({'title':each.find('h3').get_text(),'full_title': each.find(class_ = "gs_rs").get_text()})
  return art_list
  
#Retrieving URLs for all keys
def retrieveURLs(soup):
  base_url = "https://scholar.google.com/"
  keys = soup.find("div", {"id": "gs_nml"})
  key_urls = []
  for each in keys.find_all('a'):
    if int(each.get_text())<11:
      key_urls.append(str(base_url+each.get('href')))
  return key_urls

#Retrieving URL data using beautifulsoup
def getDataFromUrl(url):
  proxies = get_proxies()
  proxy_pool = cycle(proxies)
  proxy = next(proxy_pool)
  try:
    response = requests.get(url,proxies={"http": proxy, "https": proxy}).text
    b_soup = BeautifulSoup(response,'html.parser')
    return b_soup   
  except:
    #Skipping connection errors as all above created proxies won't connect to the url and gives connection issues,
    print("Skipping. Connnection error")

#Taking POS Tags
def getPOSforText(text):
  doc = nlp(text)
  for token in doc: 
    print(token.text, "-->",token.dep_,"-->", token.pos_)
  return doc

#describing question Pattern
def questPattern(token_doc):
  pattern = [{'POS':'PRON','OP':'?'},
             {'POS':'advmod'},
             {'LOWER': 'is','OP':'?'},
             {'LOWER':'are','OP':'?'},
             {'LOWER': 'was'},
             {'POS': 'NOUN','OP':'?'},
             {'POS':'PRON'}]
  matcher = Matcher(nlp.vocab).add("matching_1", None, pattern)
  matches = matcher(token_doc)
  print(len(matches)) 
  span = token_doc[matches[0][1]] 
  return span.text

#describing Problem Pattern
def probPattern(token_doc):
  pattern = [{'POS':'PRON','OP':'?'},
             {'POS':'NOUN'},
             {'LOWER': 'in','OP':'?'},
             {'LOWER':'of','OP':'?'},
             {'LOWER': 'are','OP':'?'},
             {'LOWER':'were'},
             {'POS': 'NOUN','OP':'?'},
             {'POS':'amod'}]
  matcher = Matcher(nlp.vocab).add("matching_1", None, pattern)
  matches = matcher(token_doc) 
  span = token_doc[matches] 
  return span.text

#describing fact Pattern
def factPattern(token_doc):
  pattern = [{'POS':'PRON','OP':'?'},
             {'POS':'NOUN'},
             {'LOWER': 'as','OP':'?'},
             {'LOWER':'a','OP':'?'},
             {'LOWER': 'is','OP':'?'},
             {'LOWER':'in','OP':'?'},
             {'LOWER':'the','OP':'?'},
             {'LOWER':'of'},
             {'POS': 'NOUN','OP':'?'},
             {'POS':'advmod','OP':'?'},
             {'POS':'VERB'}]

  matcher = Matcher(nlp.vocab).add("matching_1", None, pattern)
  matcher.add("matching_1", None, pattern) 
  matches = matcher(token_doc) 
  span = token_doc[matches[0][1]] 
  return span.text

#Constructing url to search article in google Scholr with keywords-Pattern Recognition Machine Learning 
query = "pattern+recognition+machine+learning"
url = 'https://scholar.google.com/scholar?q=' + query + '&ie=UTF-8&oe=UTF-8&hl=en&btnG=Search'

#Retrieving all the article titles from above functions
content = getDataFromUrl(url)
print(content)
if content:
  allTitles = getTitles(content)
  print(len(allTitles))
  '''if content:
    for each in retrieveURLs(content):
      allTitles.append(getTitles(getDataFromUrl(each)))
      print(len(allTitles))'''

#Calling above function to print the patterned texts from all the articles
for each in getArticle_dict(allTitles)[0:10]:
  print(questPattern(getPOSforText(each['full_title'])))
  print(probPattern(getPOSforText(each['full_title'])))
  print(factPattern(getPOSforText(each['full_title'])))

<!DOCTYPE doctype html>
<html><head><title>Google Scholar</title><meta content="text/html;charset=utf-8" http-equiv="Content-Type"/><meta content="IE=Edge" http-equiv="X-UA-Compatible"/><meta content="always" name="referrer"/><meta content="width=device-width,initial-scale=1,minimum-scale=1,maximum-scale=2" name="viewport"/><meta content="telephone=no" name="format-detection"/><link href="/favicon.ico" rel="shortcut icon"/><style>html,body,form,table,div,h1,h2,h3,h4,h5,h6,img,ol,ul,li,button{margin:0;padding:0;border:0;}table{border-collapse:collapse;border-width:0;empty-cells:show;}html,body{height:100%}#gs_top{position:relative;box-sizing:border-box;min-height:100%;min-width:964px;-webkit-tap-highlight-color:rgba(0,0,0,0);}#gs_top>*:not(#x){-webkit-tap-highlight-color:rgba(204,204,204,.5);}.gs_el_ph #gs_top,.gs_el_ta #gs_top{min-width:320px;}#gs_top.gs_nscl{position:fixed;width:100%;}body,td,input,button{font-size:13px;font-family:Arial,sans-serif;line-height:1.24;}body{background:#f

TypeError: ignored

In [3]:

import nltk
nltk.download('all')


[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/abc.zip.
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/alpino.zip.
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping corpora/biocreative_ppi.zip.
[nltk_data]    | Downloading package brown to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/brown.zip.
[nltk_data]    | Downloading package brown_tei to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/brown_tei.zip.
[nltk_data]    | Downloading package cess_cat to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/cess_cat.zip.
[nltk_data]    | Downloading package cess_esp to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/cess_esp.zip.
[nltk_data]    | Downloading package chat80 to /root/nltk_data...
[nltk_data]    |   Unzipp

True

## **2. Domain-specific information extraction (10 points)**

For the legal case used in the data cleaning exercise: [01-05-1 Adams v Tanner.txt](https://raw.githubusercontent.com/unt-iialab/info5731_spring2021/main/class_exercises/01-05-1%20%20Adams%20v%20Tanner.txt), use [legalNLP](https://lexpredict-lexnlp.readthedocs.io/en/latest/modules/extract/extract.html#nlp-based-extraction-methods) to extract the following inforation from the text (if the information is not exist, just print None):

(1) acts, e.g., “section 1 of the Advancing Hope Act, 1986”

(2) amounts, e.g., “ten pounds” or “5.8 megawatts”

(3) citations, e.g., “10 U.S. 100” or “1998 S. Ct. 1”

(4) companies, e.g., “Lexpredict LLC”

(5) conditions, e.g., “subject to …” or “unless and until …”

(6) constraints, e.g., “no more than”

(7) copyright, e.g., “(C) Copyright 2000 Acme”

(8) courts, e.g., “Supreme Court of New York”

(9) CUSIP, e.g., “392690QT3”

(10) dates, e.g., “June 1, 2017” or “2018-01-01”

(11) definitions, e.g., “Term shall mean …”

(12) distances, e.g., “fifteen miles”

(13) durations, e.g., “ten years” or “thirty days”

(14) geographic and geopolitical entities, e.g., “New York” or “Norway”

(15) money and currency usages, e.g., “$5” or “10 Euro”

(16) percents and rates, e.g., “10%” or “50 bps”

(17) PII, e.g., “212-212-2121” or “999-999-9999”

(18) ratios, e.g.,” 3:1” or “four to three”

(19) regulations, e.g., “32 CFR 170”

(20) trademarks, e.g., “MyApp (TM)”

(21) URLs, e.g., “http://acme.com/”

(22) addresses, e.g., “1999 Mount Read Blvd, Rochester, NY, USA, 14615”

(23) persons, e.g., “John Doe

In [2]:
#Download the lexnlp
!pip install lexnlp
from six.moves import urllib
import pandas as pd
import lexnlp

#Reading Text from the link to a string
text = ''
url = "https://raw.githubusercontent.com/unt-iialab/info5731_spring2021/main/class_exercises/01-05-1%20%20Adams%20v%20Tanner.txt"
text_file = urllib.request.urlopen(url)

for line in text_file:
  decoded_data = line.decode("utf-8").replace('\n', ' ')
  if decoded_data: text = text + decoded_data




In [3]:
#Removing Punctuations
import string
text = text.translate(str.maketrans('', '', string.punctuation))



In [4]:
#Acts
import lexnlp.extract.en.acts
acts = lexnlp.extract.en.acts.get_act_list(text)
if acts: 
  print(acts)
else:
  print('No Acts')



No Acts


In [5]:
#Accounts
import lexnlp.extract.en.amounts
print(list(lexnlp.extract.en.amounts.get_amounts(text)))



[Decimal('5.0'), Decimal('740.0'), Decimal('1843.0'), Decimal('2.0'), Decimal('1.0'), Decimal('4.0'), Decimal('2.0'), Decimal('5.0'), Decimal('1.0'), Decimal('1840.0'), Decimal('80100.0'), Decimal('30.0'), Decimal('1839.0'), Decimal('741.0'), Decimal('1840.0'), Decimal('14000.0'), Decimal('120.0'), Decimal('1.0'), Decimal('1840.0'), Decimal('3.0'), Decimal('4.0'), Decimal('1.0'), Decimal('1840.0'), Decimal('2.0'), Decimal('1.0'), Decimal('361.0'), Decimal('1.0'), Decimal('307.0'), Decimal('6.0'), Decimal('604.0'), Decimal('1.0'), Decimal('2.0'), Decimal('418.0'), Decimal('422.0'), Decimal('7.0'), Decimal('34.0'), Decimal('41.0'), Decimal('167.0'), Decimal('742.0'), Decimal('3.0'), Decimal('112.0'), Decimal('207.0'), Decimal('3.0'), Decimal('338.0'), Decimal('424.0'), Decimal('5.0'), Decimal('26.0'), Decimal('13.0'), Decimal('235.0'), Decimal('8.0'), Decimal('693.0'), Decimal('4.0'), Decimal('1821.0'), Decimal('167.0'), Decimal('2.0'), Decimal('2.0'), Decimal('216.0'), Decimal('3.0'), D

In [6]:
#Citations
import lexnlp.extract.en.citations
print(list(lexnlp.extract.en.citations.get_citations(text)))

[(1843, 'WL', 'West Law Citation', 284, None, None, None)]


In [7]:
#Companies
import lexnlp.extract.en.entities.nltk_re
print(list(lexnlp.extract.en.entities.nltk_re.get_companies(text)))

[Lehman Durr Co, (17439, 17455)]


In [9]:
#Conditions
import lexnlp.extract.en.conditions
print(list(lexnlp.extract.en.conditions.get_conditions(text)))

[('until', '5 Ala 740\r Supreme Court of Alabama\r ADAMS\r v\r TANNER AND HORTON\r June Term 1843\r Synopsis\r WRIT of Error to the Circuit Court of Sumter\r \xa0\r \r \r West Headnotes 2\r \r \r 1\r Chattel Mortgages\r Crops\r A growing crop has such an existence as to be the subjectmatter of a mortgage or other contract which passes an interest to vest in possession either immediately or at a future time\r 4 Cases that cite this headnote\r \r 2\r Creditors’ Remedies\r Lien and Priority\r Under St1821 prohibiting a levy on a crop', ''), ('until', 'it has been gathered no lien attaches in favor of a fi fa on a growing crop nor does such lien attach', ''), ('if', 'after the crop has been gathered\r 5 Cases that cite this headnote\r \r 1 This was a trial of the right of property under the statute In November 1840 an execution issued from the circuit court of Sumter at the suit of the plaintiff in error requiring the sheriff of that county to make of the goods c of Allen Harrison and othe

In [10]:
#Constraints
import lexnlp.extract.en.constraints
print(list(lexnlp.extract.en.constraints.get_constraints(text)))

[('after', '5 ala 740\r supreme court of alabama\r adams\r v\r tanner and horton\r june term 1843\r synopsis\r writ of error to the circuit court of sumter\r \xa0\r \r \r west headnotes 2\r \r \r 1\r chattel mortgages\r crops\r a growing crop has such an existence as to be the subjectmatter of a mortgage or other contract which passes an interest to vest in possession either immediately or at a future time\r 4 cases that cite this headnote\r \r 2\r creditors’ remedies\r lien and priority\r under st1821 prohibiting a levy on a crop until it has been gathered no lien attaches in favor of a fi fa on a growing crop nor does such lien attach until', ''), ('first of', 'the crop has been gathered\r 5 cases that cite this headnote\r \r 1 this was a trial of the right of property under the statute in november 1840 an execution issued from the circuit court of sumter at the suit of the plaintiff in error requiring the sheriff of that county to make of the goods c of allen harrison and others the

In [11]:
#Copy Rights

import lexnlp.extract.en.copyright
print(list(lexnlp.extract.en.copyright.get_copyright(text)))

[('©', '2019', 'Thomson Reuters No')]


In [ ]:
#Courts
import lexnlp.extract.en.dict_entities
import lexnlp.extract.en.courts
import pandas

court_df = pandas.read_csv("https://raw.githubusercontent.com/LexPredict/lexpredict-legal-dictionary/1.0.5/en/legal/us_courts.csv")
# Create config objects
court_config_data = []
for _, row in court_df.iterrows():
    x = lexnlp.extract.en.dict_entities.entity_config(row["Court ID"], row["Court Name"], 0, row["Alias"].split(";") if not pandas.isnull(row["Alias"]) else [])
    court_config_data.append(x)
for entity, alias in lexnlp.extract.en.courts.get_courts(text, court_config_data):
    print("entity=", entity)
    print("alias=", alias)


In [13]:
#Cusip
import lexnlp.extract.en.cusip
cusip = list(lexnlp.extract.en.cusip.get_cusip(text))
if cusip:
  print(cusip)
else:
  print('No Cusip')

No Cusip


In [14]:
#Dates
import lexnlp.extract.en.dates
print(list(lexnlp.extract.en.dates.get_dates(text)))

[datetime.date(2021, 6, 1), datetime.date(1840, 11, 1), datetime.date(1839, 10, 1), datetime.date(1840, 5, 1), datetime.date(1840, 9, 1), datetime.date(1840, 5, 1), datetime.date(1840, 5, 1), datetime.date(2021, 12, 1), datetime.date(2021, 12, 1), datetime.date(2021, 1, 1), datetime.date(2021, 1, 1), datetime.date(2021, 1, 1), datetime.date(2021, 3, 21), datetime.date(2021, 6, 1), datetime.date(2021, 7, 1), datetime.date(2021, 11, 1), datetime.date(1887, 5, 1)]


In [16]:
#Definitions
import lexnlp.extract.en.definitions
definitions = list(lexnlp.extract.en.definitions.get_definitions(text))
if definitions:
  print(Definitions)
else:
  print('No Definitions')

No Definitions


In [17]:
#Distances
import lexnlp.extract.en.distances
distances = list(lexnlp.extract.en.distances.get_distances(text))
if distances:
  print(Distances)
else:
  print('No Distances')

No Distances


In [18]:
#Duration
import lexnlp.extract.en.durations
print(list(lexnlp.extract.en.durations.get_durations(text)))

[('year', Decimal('6.0'), Decimal('2190.0'))]


In [ ]:
#Geographical Locations
import lexnlp.extract.en.geoentities
geo_df = pd.read_csv("https://raw.githubusercontent.com/LexPredict/lexpredict-lexnlp/master/test_data/lexnlp/extract/en/tests/test_geoentities/geoentities.csv")
geo_config_data = []
for _, row in geo_df.iterrows():
  g = entity_config(row["entity_id"], row["name"],row["category"])
  geo_config_data.append(g)
for entity, alias in lexnlp.extract.en.geoentities.get_geoentities(text, geo_config_data):
  print("entity=", entity)
  print("alias=", alias)

In [20]:
# Money
import lexnlp.extract.en.money
print(list(lexnlp.extract.en.money.get_money(text)))

[(Decimal('80100.0'), 'USD'), (Decimal('14000.0'), 'USD'), (Decimal('14000.0'), 'USD')]


In [21]:
#Percentage
import lexnlp.extract.en.percents
Percentage = list(lexnlp.extract.en.percents.get_percents(text))
if Percentage:
  print(Percentage)
else:
  print('No Percentage')

No Percentage


In [22]:
#Pii
import lexnlp.extract.en.pii
Pi = list(lexnlp.extract.en.pii.get_pii(text))
if Pi:
  print(Pi)
else:
  print('No Pi')

No Pi


In [24]:
#Ratios

import lexnlp.extract.en.ratios
Ratio = list(lexnlp.extract.en.ratios.get_ratios(text))
if Ratio:
  print(Ratio)
else:
  print('No Ratio')

No Ratio


In [25]:
#Regulation
import lexnlp.extract.en.regulations
Regulation = list(lexnlp.extract.en.regulations.get_regulations(text))
if Regulation:
  print(Regulation)
else:
  print('No Regulation')

No Regulation


In [26]:
#Trademarks
import lexnlp.extract.en.trademarks
Trademark = list(lexnlp.extract.en.trademarks.get_trademarks(text))
if Trademark:
  print(Trademark)
else:
  print('No Trademark')

No Trademark


In [27]:
#Urls
import lexnlp.extract.en.urls
Urls = list(lexnlp.extract.en.urls.get_urls(text))
if Urls:
  print(Urls)
else:
  print('No Urls')

No Urls


In [30]:
#Address
!pip install pyap
import pyap
addresses = pyap.parse(text, country='US')
for address in addresses:
  print(address)
  print(address.as_dict())

5 Ala 740 Supreme Court of Alabama ADAMS
{'full_address': '5 Ala 740 Supreme Court of Alabama ADAMS', 'full_street': '5 Ala 740 Supreme Court', 'street_number': '5', 'street_name': 'Ala 740 Supreme', 'street_type': 'Court', 'route_id': None, 'post_direction': None, 'floor': None, 'building_id': None, 'occupancy': None, 'city': 'of Alabama ADA', 'region1': 'MS', 'postal_code': None, 'country_id': 'US', 'match_start': 1, 'match_end': 42}


In [31]:
#Person
import spacy
spacy_nlp  = spacy.load('en_core_web_sm')
doc = spacy_nlp(text.strip())
person = set()
for i in doc.ents:
  entry = str(i.lemma_).lower()
  text_sample = text.replace(str(i).lower(), "")
  if i.label_ == "PERSON":
    person.add(entry.title())
person

{'Ala',
 'Ala \r  Error',
 'Allen Harrison',
 'Alston',
 'Austin',
 'Booker',
 'Burton   Harrison',
 'Case',
 'Cohen',
 'Dewey',
 'Dowl Rep',
 'Edwards',
 'Elliott',
 'Evans',
 'Harrison',
 'Jacob S Cohen',
 'Janney',
 'Lamar',
 'Lampley \r  31',
 'Lien',
 'Lien Attach',
 'Mansony',
 'Marshall',
 'Perkins',
 'Sawyer',
 'Stewart',
 'Sumter',
 'W G Jones',
 'Whipple',
 'Wood'}